In [1]:
import pypowsybl as pp
import pypowsybl.network as pn
import math

# Create network
network = pp.network.create_empty()

# Add substations
network.create_substations(
    id=['S1', 'S2', 'S3', 'S4'],
    name=['Hydro_Sub', 'Solar_Plant_CB', 'Alamoen_Sub', 'Reskjem_Sub'],
    country=['NO', 'NO', 'NO', 'NO'],
    tso=['TSO', 'TSO', 'TSO', 'TSO']
)

# Add voltage levels including 33kV for solar collection
network.create_voltage_levels(
    id=['VL1', 'VL2_33', 'VL3_33', 'VL3_132', 'VL4', 'VL5'],
    name=['Hydro_Generator', 'Solar_Plant_33kV_CB', 'Alamoen_33kV_Switchgear', 
          'Alamoen_132kV', 'Reskjem_132kV', 'Reskjem_300kV'],
    substation_id=['S1', 'S2', 'S3', 'S3', 'S4', 'S4'],
    nominal_v=[132.0, 33.0, 33.0, 132.0, 132.0, 300.0],
    topology_kind=['BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER']
)

# Add buses
network.create_buses(
    id=['B1', 'B2_33', 'B3_33', 'B3_132', 'B4', 'B5'],
    name=['Hydro_Bus', 'Solar_CB_Bus', 'Alamoen_33kV_Bus', 
          'Alamoen_132kV_Bus', 'Reskjem_132kV_Bus', 'Reskjem_300kV_Bus'],
    voltage_level_id=['VL1', 'VL2_33', 'VL3_33', 'VL3_132', 'VL4', 'VL5']
)

# Add Hydro Generator (PV Bus)
network.create_generators(
    id=['G1'],
    voltage_level_id=['VL1'],
    bus_id=['B1'],
    target_p=[100.0],          # MW
    target_q=[0.0],            # MVAr
    target_v=[132.0],          # kV
    voltage_regulator_on=[True],
    min_p=[100.0],               # MW
    max_p=[100.0],             # MW
    rated_s=[120.0],           # MVA
    energy_source=['HYDRO']
)

# Add Solar collector feeders 
for i in range(5):
    network.create_generators(
        id=[f'Feeder_{i+1}'],
        voltage_level_id=['VL2_33'],
        bus_id=['B2_33'],
        target_p=[30.0],        
        target_q=[0.0],
        target_v=[33.0],
        voltage_regulator_on=[True],
        min_p=[30.0],
        max_p=[31.0],
        rated_s=[40.0],
        energy_source=['OTHER']
    )
    
# Add Alamoen Generator (132kV)
network.create_generators(
    id=['G3'],
    name=['Alamoen PV plant'],
    voltage_level_id=['VL3_132'],      # Changed from VL3 to VL3_132
    bus_id=['B3_132'],                 # Changed from B3 to B3_132
    target_p=[103.0],                  # 103 MW
    target_q=[0.0],
    target_v=[132.0],
    voltage_regulator_on=[True],
    min_p=[103.0],
    max_p=[103.0],
    rated_s=[120.0],                   # Slightly higher than power output
    energy_source=['OTHER']
)


# Add Grid Slack Bus
network.create_generators(
    id=['GRID_SLACK'],
    voltage_level_id=['VL5'],
    bus_id=['B5'],
    target_p=[0.0],
    target_q=[0.0],
    target_v=[300.0],
    voltage_regulator_on=[True],
    min_p=[-1000.0],
    max_p=[1000.0],
    rated_s=[1000.0],
    energy_source=['OTHER']
)

# Add load at slack bus
network.create_loads(
    id=['L1'],
    name=['Grid_Load'],
    voltage_level_id=['VL5'],
    bus_id=['B5'],
    p0=[353.0],     # Total generation
    q0=[0.0]
)

# Add feeder cable parameters
def calculate_33kv_feeder_parameters(length_km):
    """33kV XLPE Cable parameters (1x500/25 CAS)
    Args:
        length_km: Cable length in kilometers
    Returns:
        Dictionary with cable parameters
    """
    r_per_km = 0.0786      # Ω/km at 90°C
    x_per_km = 0.10053      # Ω/km
    b_per_km = 9.11e-5     # S/km
    current_rating = 650    # A
    
    r_total = r_per_km * length_km
    x_total = x_per_km * length_km
    b_total = b_per_km * length_km
    return {
        'r': r_total,
        'x': x_total,
        'b': b_total,
        'i_rated': current_rating
    }


# Calculate line parameters
def calculate_line_parameters(length_km):
    r_per_km = 0.071145  # ohm/km at 20°C
    x_per_km = 0.394    # ohm/km at 50Hz
    c_per_km = 11.1e-9  # F/km
    f = 50  # Hz
    
    r_total = r_per_km * length_km
    x_total = x_per_km * length_km
    b_total = 2 * math.pi * f * c_per_km * length_km
    
    return {
        'r': r_total,
        'x': x_total,
        'b': b_total
    }

# Calculate parameters for each section
hydro_params = calculate_line_parameters(10)    # 10km for hydro
feeder_params = calculate_33kv_feeder_parameters(1.8)   # 1.8km for solar
grid_params = calculate_line_parameters(15)     # 15km for grid connection

# Add Hydro plant lines (2 x 10km)
network.create_lines(
    id=['L1_H1', 'L1_H2'],
    name=['Hydro_Line_1 (10km)', 'Hydro_Line_2 (10km)'],
    voltage_level1_id=['VL1', 'VL1'],
    bus1_id=['B1', 'B1'],
    voltage_level2_id=['VL3_132', 'VL3_132'],
    bus2_id=['B3_132', 'B3_132'],
    b1=[hydro_params['b'], hydro_params['b']],
    b2=[hydro_params['b'], hydro_params['b']],
    g1=[0.0, 0.0],
    g2=[0.0, 0.0],
    r=[hydro_params['r'], hydro_params['r']],
    x=[hydro_params['x'], hydro_params['x']],
)

# Add Aalamoent transformer (33/132 kV) for solar
network.create_2_windings_transformers(
    id=['TR1_A', 'TR1_B'],
    name=['Alamoen_Transformer_A', 'Alamoen_Transformer_B'],
    voltage_level1_id=['VL3_33', 'VL3_33'],      #  (Alamoen 33kV)
    voltage_level2_id=['VL3_132', 'VL3_132'],    # (Alamoen 132kV)
    bus1_id=['B3_33', 'B3_33'],                  #  (Alamoen 33kV bus)
    bus2_id=['B3_132', 'B3_132'],                #  (Alamoen 132kV bus)
    rated_u1=[33.0, 33.0],                      
    rated_u2=[132.0, 132.0],
    rated_s=[100.0, 100.0],
    r=[0.725, 0.725],
    x=[21.75, 21.75],
    g=[5.7e-6, 5.7e-6],
    b=[9.92e-6, 9.92e-6]
)

# Add 33kV feeder lines from solar plant to Alamoen
for i in range(5):
    network.create_lines(
        id=[f'F{i+1}_33KV'],
        name=[f'Solar_Feeder_{i+1}_33kV_1.8km'],
        voltage_level1_id=['VL2_33'],    # From CB bus
        voltage_level2_id=['VL3_33'],    # To Alamoen switchgear
        bus1_id=['B2_33'],              # From CB bus
        bus2_id=['B3_33'],              # To Alamoen bus
        r=[feeder_params['r']],
        x=[feeder_params['x']],
        g1=[0.0],
        b1=[feeder_params['b']],
        g2=[0.0],
        b2=[feeder_params['b']]
    )
# Add Grid connection lines (2 x 15km)
network.create_lines(
    id=['L3_G1', 'L3_G2'],
    name=['Grid_Line_1 (15km)', 'Grid_Line_2 (15km)'],
    voltage_level1_id=['VL3_132', 'VL3_132'],   
    voltage_level2_id=['VL4', 'VL4'],
    bus1_id=['B3_132', 'B3_132'],                
    bus2_id=['B4', 'B4'],
    r=[grid_params['r'], grid_params['r']],
    x=[grid_params['x'], grid_params['x']],
    g1=[0.0, 0.0],
    b1=[grid_params['b'], grid_params['b']],
    g2=[0.0, 0.0],
    b2=[grid_params['b'], grid_params['b']]
)

# Add main grid transformers (3 × 160MVA, 132/300 kV)
network.create_2_windings_transformers(
    id=['T1_A', 'T1_B', 'T1_C'],
    name=['Grid_Transformer_A', 'Grid_Transformer_B', 'Grid_Transformer_C'],
    voltage_level1_id=['VL4', 'VL4', 'VL4'],
    voltage_level2_id=['VL5', 'VL5', 'VL5'],
    bus1_id=['B4', 'B4', 'B4'],
    bus2_id=['B5', 'B5', 'B5'],
    rated_u1=[132.0, 132.0, 132.0],       # Primary voltage (kV)
    rated_u2=[300.0, 300.0, 300.0],       # Secondary voltage (kV)
    rated_s=[160.0, 160.0, 160.0],        # Rated power (MVA)
    r=[1.76, 1.76, 1.76],                 # Resistance referred to HV side (Ω)
    x=[70.4, 70.4, 70.4],                 # Reactance referred to HV side (Ω)
    g=[1.78e-6, 1.78e-6, 1.78e-6],       # Conductance (S)
    b=[3.08e-6, 3.08e-6, 3.08e-6]        # Susceptance (S)
)

# Run power flow analysis
print("\n=== Power Flow Analysis ===")
results = pp.loadflow.run_ac(network)

print("\nConvergence Status:")
for result in results:
    print(f"Status: {result.status}")
    print(f"Details: {result.status_text}")

# Update system configuration print
print("\n=== System Configuration ===")
print("1. Solar Plant (33kV Collection):")
print("- 5 feeders x 30.0 MW = 150 MW")

print("\n2. Aalamoen Substation:")
print("- Step-up transformer: 2 × 100MVA (33/132 kV)")
print("- Transmission line: 1.8 km at 132kV")

print("\n2. Hydro Plant:")
print("- Generation: 100 MW")
print("- Voltage: 132 kV")
print("- Transmission: 2 x 10 km lines")

print("\n3. Alamoen Plant:")
print("- Generation: 103 MW")
print("- Voltage: 132 kV")

print("\n4. Grid Connection:")
print("- 2 x 15 km lines at 132 kV")
print("- Main transformers: 3 × 160MVA (132/300 kV)")

#Slack generator
gen_results = network.get_generators()
slack_gen = gen_results.loc['GRID_SLACK']
print("\n=== Slack Bus Analysis ===")
print(f"Location: {slack_gen['name']}")
print(f"Voltage: {slack_gen['target_v']:.2f} kV")
print(f"Power absorbed: {abs(slack_gen['p']):.2f} MW")      # Changed from target_p to p
print(f"Reactive power: {slack_gen['q']:.2f} MVAR")         # Changed from target_q to q
print("\nTarget values:")
print(f"Target P: {slack_gen['target_p']:.2f} MW")
print(f"Target Q: {slack_gen['target_q']:.2f} MVAR")


# Get bus voltages and calculate voltage drops
bus_results = network.get_buses()
print("\n=== Voltage Profile and Drops ===")
nominal_voltages = {
    'VL1': 132.0,     # Hydro
    'VL2_33': 33.0,   # Solar collection
    'VL3_33': 33.0,   # Alamoen 33kV
    'VL3_132': 132.0, # Alamoen 132kV
    'VL4': 132.0,     # Reskjem 132kV
    'VL5': 300.0      # Reskjem 300kV
}

for bus_id in bus_results.index:
    v_mag = bus_results.loc[bus_id, 'v_mag']
    v_angle = bus_results.loc[bus_id, 'v_angle']
    vl_id = bus_results.loc[bus_id, 'voltage_level_id']
    nominal_v = nominal_voltages[vl_id]
    voltage_deviation = ((v_mag - nominal_v)/nominal_v) * 100
    
    print(f"\nBus: {bus_results.loc[bus_id, 'name']}")
    print(f"Actual Voltage: {v_mag:.2f} kV")
    print(f"Nominal Voltage: {nominal_v:.2f} kV")
    print(f"Voltage Deviation: {voltage_deviation:+.2f}%")
    print(f"Angle: {v_angle:.4f}°")

# Analyze transmission path voltage drops
print("\n=== Transmission Path Voltage Drop Analysis ===")

print("\n1. Solar Plant Feeders (33kV, 1.8km):")
for i in range(5):
    feeder_id = f'F{i+1}_33KV'
    feeder = network.get_lines().loc[feeder_id]
    sending_v = bus_results.loc[feeder['bus1_id'], 'v_mag']
    receiving_v = bus_results.loc[feeder['bus2_id'], 'v_mag']
    voltage_drop = sending_v - receiving_v
    current = feeder['i1']
    power = abs(feeder['p1'])
    losses = abs(feeder['p1'] + feeder['p2'])
    
    print(f"\nFeeder {i+1}:")
    print(f"Power Flow: {power:.2f} MW")
    print(f"Current: {current:.2f} A")
    print(f"Sending End Voltage: {sending_v:.2f} kV")
    print(f"Receiving End Voltage: {receiving_v:.2f} kV")
    print(f"Voltage Drop: {voltage_drop:.2f} kV ({(voltage_drop/sending_v*100):+.2f}%)")
    print(f"Losses: {losses:.2f} MW ({(losses/power*100):.2f}%)")
    
print("\n=== Transformer Analysis ===")
transformer_results = network.get_2_windings_transformers()
print("\n2. Alamoen Transformers (33/132 kV):")
for transformer_id in ['TR1_A', 'TR1_B']:
    transformer = transformer_results.loc[transformer_id]
    lv_voltage = bus_results.loc[transformer['bus1_id'], 'v_mag']
    hv_voltage = bus_results.loc[transformer['bus2_id'], 'v_mag']
    
    p1 = transformer['p1']
    p2 = transformer['p2']
    losses = abs(p1 + p2)
    loading = (abs(p1)/float(transformer['rated_s'])) * 100
    
    print(f"\nTransformer {transformer_id}:")
    print(f"LV Side (33kV): {lv_voltage:.2f} kV")
    print(f"HV Side (132kV): {hv_voltage:.2f} kV")
    print(f"Power Flow: {abs(p1):.2f} MW")
    print(f"Losses: {losses:.2f} MW")
    print(f"Loading: {loading:.1f}%")

print("\n6. Main Transformers (132/300 kV):")

# Analyze each main transformer
for transformer_id in ['T1_A', 'T1_B', 'T1_C']:
    main_transformer = transformer_results.loc[transformer_id]
    lv_voltage = bus_results.loc[main_transformer['bus1_id'], 'v_mag']
    hv_voltage = bus_results.loc[main_transformer['bus2_id'], 'v_mag']
    
    print(f"\nTransformer {transformer_id}:")
    print(f"LV Side: {lv_voltage:.2f} kV")
    print(f"HV Side: {hv_voltage:.2f} kV")
    print(f"Transformation Ratio: {(hv_voltage/lv_voltage):.4f}")
    
    # Calculate loading and losses
    p1 = main_transformer['p1']
    p2 = main_transformer['p2']
    losses = abs(p1 + p2)
    loading = (abs(p1)/float(main_transformer['rated_s'])) * 100
    print(f"Loading: {loading:.1f}%")
    print(f"Losses: {losses:.2f} MW")

# Calculate total transformer stats
total_main_transformer_losses = sum(abs(transformer_results.loc[tid, 'p1'] + 
                                      transformer_results.loc[tid, 'p2']) 
                                  for tid in ['T1_A', 'T1_B', 'T1_C'])

print(f"\nTotal Main Transformer System:")
print(f"Total Losses: {total_main_transformer_losses:.2f} MW")
average_loading = sum(abs(transformer_results.loc[tid, 'p1'])/
                     float(transformer_results.loc[tid, 'rated_s']) * 100 
                     for tid in ['T1_A', 'T1_B', 'T1_C'])/3
print(f"Average Loading: {average_loading:.1f}%")

print("\n4. Hydro Plant Lines (2 x 10km):")
for line_id in ['L1_H1', 'L1_H2']:
    line_results = network.get_lines()
    sending_bus = line_results.loc[line_id, 'bus1_id']
    receiving_bus = line_results.loc[line_id, 'bus2_id']
    v1 = bus_results.loc[sending_bus, 'v_mag']
    v2 = bus_results.loc[receiving_bus, 'v_mag']
    voltage_drop = v1 - v2
    print(f"\nLine {line_id}:")
    print(f"Sending End: {v1:.2f} kV")
    print(f"Receiving End: {v2:.2f} kV")
    print(f"Voltage Drop: {voltage_drop:.2f} kV ({(voltage_drop/v1*100):+.2f}%)")

print("\n5. Grid Connection Lines (2 x 15km):")
for line_id in ['L3_G1', 'L3_G2']:
    line_results = network.get_lines()
    sending_bus = line_results.loc[line_id, 'bus1_id']
    receiving_bus = line_results.loc[line_id, 'bus2_id']
    v1 = bus_results.loc[sending_bus, 'v_mag']
    v2 = bus_results.loc[receiving_bus, 'v_mag']
    voltage_drop = v1 - v2
    print(f"\nLine {line_id}:")
    print(f"Sending End: {v1:.2f} kV")
    print(f"Receiving End: {v2:.2f} kV")
    print(f"Voltage Drop: {voltage_drop:.2f} kV ({(voltage_drop/v1*100):+.2f}%)")


# Get line flows
line_results = network.get_lines()
print("\nLine Power Flows:")
for line_id in line_results.index:
    print(f"\nLine {line_results.loc[line_id, 'name']}:")
    p1 = line_results.loc[line_id, 'p1']
    q1 = line_results.loc[line_id, 'q1']
    p2 = line_results.loc[line_id, 'p2']
    q2 = line_results.loc[line_id, 'q2']
    i1 = line_results.loc[line_id, 'i1']
    losses = abs(p1 + p2)
    print(f"Power Flow: {p1:.2f} MW + j{q1:.2f} MVAr → {-p2:.2f} MW + j{-q2:.2f} MVAr")
    print(f"Current: {i1:.2f} A")
    print(f"Losses: {losses:.2f} MW")

# Calculate total system losses
feeder_losses = sum(abs(network.get_lines().loc[f'F{i+1}_33KV']['p1'] + network.get_lines().loc[f'F{i+1}_33KV']['p2']) for i in range(5))


alamoen_transformer_losses = sum(abs(transformer_results.loc[tid, 'p1'] + 
                                   transformer_results.loc[tid, 'p2']) 
                               for tid in ['TR1_A', 'TR1_B'])

total_132kv_losses = sum(abs(line_results.loc[line_id, 'p1'] + 
                            line_results.loc[line_id, 'p2'])
                        for line_id in ['L1_H1', 'L1_H2', 'L3_G1', 'L3_G2'])

print("\n=== Total System Losses Summary ===")
print(f"33kV Feeder Losses: {feeder_losses:.2f} MW")
print(f"33/132kV Transformer Losses: {alamoen_transformer_losses:.2f} MW")
print(f"132kV System Losses: {total_132kv_losses:.2f} MW")
print(f"132/300kV Transformer Losses: {total_main_transformer_losses:.2f} MW")
print(f"Total System Losses: {(feeder_losses + alamoen_transformer_losses + total_132kv_losses + total_main_transformer_losses):.2f} MW")

# Create SLD
# Create base SLD parameters
param = pn.SldParameters(
    use_name=True,
    center_name=True,
    diagonal_label=False,
    nodes_infos=True,
    topological_coloring=True,
    component_library="FlatDesign",
    active_power_unit="MW",
    reactive_power_unit="MVAR",
    display_current_feeder_info=True
)

# Layout 1: 2x2 Matrix
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], 
     ['S3', 'S4']], 
    'sld_2x2_matrix_aalamoen_33KV.svg',
    parameters=param
)

# Layout 2: Vertical Flow
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1'], 
     ['S2'],
     ['S3'],
     ['S4']], 
    'sld_vertical_aalamoen_33KV.svg',
    parameters=param
)

# Layout 3: Horizontal Flow
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2', 'S3', 'S4']], 
    'sld_horizontal_aalamoen_33KV.svg',
    parameters=param
)

# Layout 4: L-shaped
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2', 'S3'], 
     ['', '', 'S4']], 
    'sld_L_shape_aalamoen_33KV.svg',
    parameters=param
)

# Layout 6: Separated Generation and Grid
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2', ''], 
     ['', 'S3', 'S4']], 
    'sld_separated_aalamoen_33KV.svg',
    parameters=param
)

# Layout 7: Cascade
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1'], 
     ['S2', 'S3'], 
     ['', 'S4']], 
    'sld_cascade_aalamoen_33KV.svg',
    parameters=param
)

# Also try different component libraries
param_flat = pn.SldParameters(
    use_name=True,
    center_name=True,
    diagonal_label=False,
    nodes_infos=True,
    topological_coloring=True,
    component_library="FlatDesign",
    active_power_unit= "mw",
    reactive_power_unit="MVAR",
    display_current_feeder_info=True
)

param_convergence = pn.SldParameters(
    use_name=True,
    center_name=True,
    diagonal_label=False,
    nodes_infos=True,
    topological_coloring=True,
    component_library="Convergence",
    active_power_unit="MW",
    reactive_power_unit="MVAR",
    display_current_feeder_info=True
)

# Save with different component libraries
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], ['S3', 'S4']], 
    'sld_flat_design_aalamoen_33KV.svg',
    parameters=param_flat
)

network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], ['S3', 'S4']], 
    'sld_convergence_aalamoen_33KV.svg',
    parameters=param_convergence
)

print("Generated multiple SLD layouts:")



=== Power Flow Analysis ===

Convergence Status:
Status: ComponentStatus.CONVERGED
Details: Converged

=== System Configuration ===
1. Solar Plant (33kV Collection):
- 5 feeders x 30.0 MW = 150 MW

2. Aalamoen Substation:
- Step-up transformer: 2 × 100MVA (33/132 kV)
- Transmission line: 1.8 km at 132kV

2. Hydro Plant:
- Generation: 100 MW
- Voltage: 132 kV
- Transmission: 2 x 10 km lines

3. Alamoen Plant:
- Generation: 103 MW
- Voltage: 132 kV

4. Grid Connection:
- 2 x 15 km lines at 132 kV
- Main transformers: 3 × 160MVA (132/300 kV)

=== Slack Bus Analysis ===
Location: 
Voltage: 300.00 kV
Power absorbed: 0.00 MW
Reactive power: -56.79 MVAR

Target values:
Target P: 0.00 MW
Target Q: 0.00 MVAR

=== Voltage Profile and Drops ===

Bus: Hydro_Generator_0
Actual Voltage: 132.00 kV
Nominal Voltage: 132.00 kV
Voltage Deviation: +0.00%
Angle: 9.4921°

Bus: Solar_Plant_33kV_CB_0
Actual Voltage: 33.00 kV
Nominal Voltage: 33.00 kV
Voltage Deviation: +0.00%
Angle: 14.6747°

Bus: Alamoen_33

In [2]:
# Loss Analysis
print("\n=== Loss Analysis for Complete System ===")

# Calculate total generation
total_generation = 0
gen_results = network.get_generators()
print("\nGeneration Summary:")
for gen_id in gen_results.index:
    if gen_id != 'GRID_SLACK':
        power = abs(gen_results.loc[gen_id, 'target_p'])
        total_generation += power
        print(f"Generator {gen_id}: {power:.2f} MW")
print(f"Total Generation: {total_generation:.2f} MW")

# 33kV Feeder Analysis
print("\n33kV Feeder System Losses:")
feeder_losses = 0
total_feeder_current = 0
line_results = network.get_lines()

for i in range(5):
    line_id = f'F{i+1}_33KV'
    line = line_results.loc[line_id]
    line_loss = abs(line['p1'] + line['p2'])
    feeder_losses += line_loss
    current = line['i1']
    power = abs(line['p1'])
    loading = (current/650.0) * 100  # 650A rating
    
    print(f"\nFeeder {i+1}:")
    print(f"Power Flow: {power:.2f} MW")
    print(f"Current: {current:.2f} A")
    print(f"Loading: {loading:.1f}% of rating")
    print(f"Losses: {line_loss:.3f} MW")
    print(f"Loss percentage: {(line_loss/power*100):.3f}%")

print(f"\nTotal 33kV Feeder Statistics:")
print(f"Total Feeder Losses: {feeder_losses:.3f} MW")

# Calculate 132kV system losses
# Hydro lines losses
hydro_losses = 0
for line_id in ['L1_H1', 'L1_H2']:
    line = line_results.loc[line_id]
    line_loss = abs(line['p1'] + line['p2'])
    hydro_losses += line_loss
    print(f"\nHydro Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Power Flow: {abs(line['p1']):.2f} MW")
    print(f"Losses: {line_loss:.2f} MW")
    print(f"Loss percentage: {(line_loss/abs(line['p1'])*100):.3f}%")

# Grid lines losses
grid_losses = 0
for line_id in ['L3_G1', 'L3_G2']:
    line = line_results.loc[line_id]
    line_loss = abs(line['p1'] + line['p2'])
    grid_losses += line_loss
    print(f"\nGrid Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Power Flow: {abs(line['p1']):.2f} MW")
    print(f"Losses: {line_loss:.2f} MW")
    print(f"Loss percentage: {(line_loss/abs(line['p1'])*100):.3f}%")

# Transformer losses
print("\nAlamoen Transformers (33/132 kV):")
transformer_results = network.get_2_windings_transformers()
alamoen_transformer_losses = 0
for trans_id in ['TR1_A', 'TR1_B']:
    transformer = transformer_results.loc[trans_id]
    transformer_loss = abs(transformer['p1'] + transformer['p2'])
    alamoen_transformer_losses += transformer_loss
    loading = (abs(transformer['p1'])/float(transformer['rated_s'])) * 100
    print(f"\n{transformer['name']}:")
    print(f"Power Flow: {abs(transformer['p1']):.2f} MW")
    print(f"Losses: {transformer_loss:.2f} MW")
    print(f"Loss percentage: {(transformer_loss/abs(transformer['p1'])*100):.3f}%")
    print(f"Loading: {loading:.1f}%")

print("\nMain Grid Transformers (132/300 kV):")
main_transformer_losses = 0
for trans_id in ['T1_A', 'T1_B', 'T1_C']:
    transformer = transformer_results.loc[trans_id]
    transformer_loss = abs(transformer['p1'] + transformer['p2'])
    main_transformer_losses += transformer_loss
    loading = (abs(transformer['p1'])/float(transformer['rated_s'])) * 100
    print(f"\n{transformer['name']}:")
    print(f"Power Flow: {abs(transformer['p1']):.2f} MW")
    print(f"Losses: {transformer_loss:.2f} MW")
    print(f"Loss percentage: {(transformer_loss/abs(transformer['p1'])*100):.3f}%")
    print(f"Loading: {loading:.1f}%")

# Total losses summary
total_132kv_losses = hydro_losses + grid_losses
total_transformer_losses = alamoen_transformer_losses
total_system_losses = feeder_losses + total_132kv_losses + total_transformer_losses

print("\n=== Total System Losses Summary ===")
print(f"33kV Feeder Losses: {feeder_losses:.3f} MW")
print(f"132kV System Losses: {total_132kv_losses:.3f} MW")
print(f"Alamoen Transformer Losses (33/132kV): {alamoen_transformer_losses:.3f} MW")
print(f"Main Transformer Losses (132/300kV): {main_transformer_losses:.3f} MW")
print(f"Total System Losses: {total_system_losses:.3f} MW")
print(f"System Efficiency: {((total_generation - total_system_losses)/total_generation * 100):.3f}%")


=== Loss Analysis for Complete System ===

Generation Summary:
Generator G1: 100.00 MW
Generator Feeder_1: 30.00 MW
Generator Feeder_2: 30.00 MW
Generator Feeder_3: 30.00 MW
Generator Feeder_4: 30.00 MW
Generator Feeder_5: 30.00 MW
Generator G3: 103.00 MW
Total Generation: 353.00 MW

33kV Feeder System Losses:

Feeder 1:
Power Flow: 31.00 MW
Current: 542.66 A
Loading: 83.5% of rating
Losses: 0.125 MW
Loss percentage: 0.403%

Feeder 2:
Power Flow: 31.00 MW
Current: 542.66 A
Loading: 83.5% of rating
Losses: 0.125 MW
Loss percentage: 0.403%

Feeder 3:
Power Flow: 31.00 MW
Current: 542.66 A
Loading: 83.5% of rating
Losses: 0.125 MW
Loss percentage: 0.403%

Feeder 4:
Power Flow: 31.00 MW
Current: 542.66 A
Loading: 83.5% of rating
Losses: 0.125 MW
Loss percentage: 0.403%

Feeder 5:
Power Flow: 31.00 MW
Current: 542.66 A
Loading: 83.5% of rating
Losses: 0.125 MW
Loss percentage: 0.403%

Total 33kV Feeder Statistics:
Total Feeder Losses: 0.625 MW

Hydro Line Hydro_Line_1 (10km):
Current: 222.